scale05.eecs.yorku.ca, 

* 8881, d9559ffb6837ac01591fd79f187ae19c0f01304b8529b1c7

* 8888, 5985df79388a705731a8d888a728663c3457e6449e00dc0a

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import pandas as pd
# https://pythonprogramming.net/cnn-tensorflow-convolutional-nerual-network-machine-learning-tutorial/
# other cnn codes: https://www.datacamp.com/community/tutorials/cnn-tensorflow-python

MODEL_SAVE_PATH = "CNN_model/"
MODEL_NAME = "cnn_model"
SUMMARY_PATH = "CNN_Logs"
KEEP_RATE = 0.8

N_CLASSES = 10
BATCH_SIZE = 128

def performance_metrics1(confusion_matrix):
    #https://blog.csdn.net/sihailongwang/article/details/77527970
    #好像不一定对。值得斟酌。
    
    accu = [0 for i in range(N_CLASSES)]
    column = [0 for i in range(N_CLASSES)]
    line = [0 for i in range(N_CLASSES)]
    accuracy = 0
    recall = 0
    precision = 0
    for i in range(0, N_CLASSES):
        accu[i] = confusion_matrix[i][i]
    for i in range(0, N_CLASSES):
       for j in range(0, N_CLASSES):
           column[i]+=confusion_matrix[j][i]
    for i in range(0, N_CLASSES):
       for j in range(0, N_CLASSES):
           line[i]+=confusion_matrix[i][j]
    # for i in range(0, N_CLASSES):
    #     accuracy += float(accu[i])/len_labels_all
    for i in range(0, N_CLASSES):
        if column[i] != 0:
            recall+=float(accu[i])/column[i]
    recall = recall /  N_CLASSES
    for i in range(0, N_CLASSES):
        if line[i] != 0:
            precision+=float(accu[i])/line[i]
    precision = precision /  N_CLASSES
    f1_score = (2 * (precision * recall)) / (precision + recall)
    print("Average Precision(AP): %f, \n"
          "Average Recall(AR): %f, \n"
          "F1 of AP and AR: %f"
          %(precision, recall, f1_score) 
    )
    return precision, recall, f1_score

def performance_metrics(confusion_matrix):
    # 上一个方法precision和recall在行列关系上好像有点奇怪，可能弄反了。
    # 而且计算的是各个类的平均值。
    # 不过基本思想是对的，在弄清楚弄反与否之前暂时不要删掉前一个方法。
    # 这个方法基本上是正统的计算方法了。而且会给出每一个类别的所有metrics。
    # 行列代表的含义和该链接所述的相同：https://zhuanlan.zhihu.com/p/33273532
    accu = [0 for i in range(N_CLASSES)]
    column = [0 for i in range(N_CLASSES)]
    line = [0 for i in range(N_CLASSES)]
    for i in range(0, N_CLASSES):
        accu[i] = confusion_matrix[i][i]
    for i in range(0, N_CLASSES):
       for j in range(0, N_CLASSES):
           column[i]+=confusion_matrix[j][i]
    for i in range(0, N_CLASSES):
       for j in range(0, N_CLASSES):
           line[i]+=confusion_matrix[i][j]
    total_num = sum(line)
    
    for i in range(N_CLASSES):
        TP = accu[i]
        FP = column[i] - accu[i]
        FN = line[i] - accu[i]
        TN = total_num - TP - FP - FN
        
        recall = float(TP) / (TP + FN)
        precision = float(TP) / (TP + FP)
        f1_score = (2 * (precision * recall)) / (precision + recall)
        print("For class %d: \n"
            "Average Precision(AP): %f, \n"
            "Average Recall(AR): %f, \n"
            "F1 of AP and AR: %f \n\n"
            %(i, precision, recall, f1_score) 
        )
        

def get_Batch1(data, label, batch_size):
#https://blog.csdn.net/sinat_35821976/article/details/82668555 
    x_batch = data.sample(batch_size)
    y_batch = label.loc[list(x_batch.index)]
    return x_batch, y_batch

def get_Batch(data, label, batch_size):
#https://blog.csdn.net/sinat_35821976/article/details/82668555 
    print(data.shape, label.shape)
    input_queue = tf.train.slice_input_producer([data, label], num_epochs=1, shuffle=True, capacity=32 ) 
    x_batch, y_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=32, allow_smaller_final_batch=False)
    return x_batch, y_batch

class Dataset:
    data = None
    label = None
    def __init__(self, path = "train.csv"):
        df = pd.read_csv(path,header=0)
        cols = df.columns.values.tolist()
        cols.pop(0)
        self.data = df[cols]
        self.label = pd.get_dummies(df['label'])

    def bi_split(self, percentage = 0.3):
        length = round(len(self.data) * percentage)
        
        return self.data[0:length], self.label[0:length], \
               self.data[length + 1:], self.label[length + 1:]
    
    def tri_split(self, p1= 0.3, p2= 0.9):
        l1 = round(len(self.data) * p1)
        l2 = round(len(self.data) * p2)
        
        return self.data[0:l1], self.label[0:l1], \
                self.data[l1+1:l2], self.label[l1+1:l2], \
               self.data[l2+1:], self.label[l2+1:]

In [2]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def convolutional_neural_network(x):
    weights = {'W_conv1': tf.Variable(tf.random_normal([5, 5, 1, 32]), 
                                      name= 'W_conv1'),
               'W_conv2': tf.Variable(tf.random_normal([5, 5, 32, 64]), 
                                      name = 'W_conv2'),
               'W_fc': tf.Variable(tf.random_normal([7 * 7 * 64, 1024]), 
                                   name= 'W_fc'),
               'out': tf.Variable(tf.random_normal([1024, N_CLASSES]), 
                                  name= 'out')}

    biases = {'b_conv1': tf.Variable(tf.random_normal([32]),
                                     name= 'b_conv1'),
              'b_conv2': tf.Variable(tf.random_normal([64]), 
                                     name= 'b_conv2'),
              'b_fc': tf.Variable(tf.random_normal([1024]), 
                                  name= 'b_fc'),
              'out': tf.Variable(tf.random_normal([N_CLASSES]), 
                                 name= 'out')}

    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)

    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2, [-1, 7 * 7 * 64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    fc = tf.nn.dropout(fc, KEEP_RATE)

    output = tf.add(tf.matmul(fc, weights['out']), 
                    biases['out'], 
                    name= "predict_output")

    return output


In [3]:
tnd1, tnl1, tnd2, tnl2, tstd, tstl = Dataset().tri_split(0.45, 0.9)

In [4]:
def train_neural_network1(train_data, train_label, test_data, test_label):

    x = tf.placeholder(tf.float32, [None, 784], name = "x-input")
    y = tf.placeholder(tf.float32, name = "y-input")

    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits= prediction, 
            labels= y),
        name= "cost"
    )
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 3
    saver = tf.train.Saver()

    with tf.control_dependencies([optimizer]):
        train_op = tf.no_op(name='train')

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(len(train_data) / BATCH_SIZE)):
                epoch_x, epoch_y = get_Batch1(train_data, 
                                             train_label, 
                                             BATCH_SIZE)
                _, c = sess.run([train_op, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_data, y: test_label}))
        
        # 普通ckpt保存
        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME))
        
        # 导出可读模型
        saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME) + ".json", as_text=True)
        
        # visualize the graph
        writer = tf.summary.FileWriter(os.path.join(SUMMARY_PATH, "original"), 
                               tf.get_default_graph())
        writer.close()
        
        # confusion_matrix,用来计算model performance
        confusion_matrix = tf.contrib.metrics.confusion_matrix(
            tf.argmax(y, 1), 
            tf.argmax(prediction, 1)
        )
        cm = confusion_matrix.eval({x: test_data, 
                                 y: test_label}
                                   )
        print(cm)
        return cm

In [5]:
if not os.path.exists(MODEL_SAVE_PATH):
    os.mkdir(MODEL_SAVE_PATH)
print(os.getcwd())

# mnist dataset: https://blog.csdn.net/gaoyueace/article/details/79056085
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
#train_neural_network(mnist)

cm_origin = train_neural_network1(tnd1, tnl1, tstd, tstl)
performance_metrics(cm_origin)


/notebooks
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch 0 completed out of 3 loss: 470498489.75


Epoch 1 completed out of 3 loss: 90129047.265625


Epoch 2 completed out of 3 loss: 50623827.7421875


Accuracy: 0.9097404


[[432   3   1   0   1   6   6   1   1   3]
 [  0 446   2   0   0   1   3   3   1   2]
 [  9   2 346   8   1   4   5   4  10   3]
 [  3   4   7 405   0  12   5   3   3   6]
 [  1   2   2   2 403   1   4   3   4  16]
 [  5   1   1   7   2 321   5   2   6   4]
 [  2   1   2   0   3   4 400   0   1   0]
 [  2   0   5   5   2   3   0 390   3  11]
 [  2  13  10   7   3   7   2   2 342   9]
 [  1   0   2   8  11   2   1  14   0 385]]
For class 0: 
Average Precision(AP): 0.945295, 
Average Recall(AR): 0.951542, 
F1 of AP and AR: 0.948408 


For class 1: 
Average Precision(AP): 0.944915, 
Average Recall(AR): 0.973799, 
F1 of AP and AR: 0.959140 


For class 2: 
Average Precision(AP): 0.915344, 
Average Recall(AR): 0.882653, 
F1 of AP and AR: 0.898701 


For class 3: 
Average Precision(AP): 0.916290, 
Average Recall(AR): 0.904018, 
F1 of AP and AR: 0.910112 


For class 4: 
Average Precision(AP): 0.946009, 
Average Recall(AR): 0.920091, 
F1 of AP and AR: 0.932870 


For class 5: 
Average Precisi

In [6]:
from tensorflow.python.tools import inspect_checkpoint as chkp
print(os.getcwd())
chkp.print_tensors_in_checkpoint_file(os.path.join(MODEL_SAVE_PATH, MODEL_NAME), 
                                      tensor_name='', 
                                      all_tensors=True)


/notebooks
tensor_name:  W_conv1
[[[[ 1.72033453e+00 -5.64643025e-01  1.14844549e+00 -1.28576982e+00
     1.91119313e+00 -2.34687114e+00 -7.31668249e-02  2.82263428e-01
    -2.89480299e-01 -6.49915993e-01  8.23776305e-01  5.99965334e-01
     1.10700512e+00 -6.53695643e-01 -1.01028085e-01  1.22033882e+00
    -1.38407424e-02  1.15286767e-01 -1.84193897e+00 -9.09164548e-01
    -2.03737244e-02  8.58729184e-02 -1.04101133e+00  5.05697131e-01
    -9.80941951e-01  2.27003857e-01  1.00259328e+00 -5.37812710e-01
     8.08088481e-01  7.06568584e-02 -3.15401107e-01 -3.36060911e-01]]

  [[-2.02778772e-01  1.15369511e+00  5.13435006e-01 -2.51916319e-01
    -1.02437961e+00  1.00801730e+00 -2.09386885e-01  6.13783538e-01
    -7.65852630e-01  1.77625239e-01  1.06333345e-01 -8.10104728e-01
     1.65445471e+00  4.05952513e-01 -3.68725091e-01  4.26862210e-01
     6.99591517e-01  3.27045530e-01 -6.55826852e-02  6.35149851e-02
     8.02438617e-01 -4.23412859e-01 -3.02733749e-01  1.83234215e+00
     1.34729

In [8]:
def restore_and_retrain_model1(input_checkpoint, 
                               train_data, train_label, 
                               test_data, test_label):

    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', 
                                       clear_devices=True)
    # graph = tf.get_default_graph()  # 获得默认的图
    # input_graph_def = graph.as_graph_def()  # 返回一个序列化的图代表当前的图
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)  # 恢复图并得到数据
        graph = tf.get_default_graph()
        cost = graph.get_tensor_by_name("cost:0")
        x = graph.get_tensor_by_name("x-input:0")
        y = graph.get_tensor_by_name("y-input:0")
        prediction = graph.get_tensor_by_name("predict_output:0")
        train_op = graph.get_operation_by_name("train")
        hm_epochs = 2
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(len(train_data) / BATCH_SIZE)):
                epoch_x, epoch_y = get_Batch1(train_data, train_label, 
                                              BATCH_SIZE)
                _, c = sess.run([train_op, cost], feed_dict={x: 
                                                                 epoch_x, 
                                                             y: 
                                                                 epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_data, y: test_label}))

        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_retrained"))
        saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restrained") + ".json", as_text=True)

        # visualize the graph
        writer = tf.summary.FileWriter(os.path.join(SUMMARY_PATH, "retrained"), 
                               tf.get_default_graph())
        writer.close()

        confusion_matrix = tf.contrib.metrics.confusion_matrix(
            tf.argmax(y, 1), 
            tf.argmax(prediction, 1)
        )
        cm = confusion_matrix.eval({x: test_data, 
                                 y: test_label}
                                   )
        print(cm)
        return cm

In [9]:
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
cm_retrain = restore_and_retrain_model1("CNN_model/cnn_model", tnd2, tnl2, tstd, tstl)
performance_metrics(cm_retrain)

INFO:tensorflow:Restoring parameters from CNN_model/cnn_model


Epoch 0 completed out of 2 loss: 41756619.9375


Epoch 1 completed out of 2 loss: 28010026.001953125


Accuracy: 0.9349845


[[439   1   4   0   1   0   4   0   4   1]
 [  1 450   1   2   1   0   2   0   1   0]
 [  5   6 354   6   3   0   0   3  15   0]
 [  1   9   5 399   2  14   1   3  12   2]
 [  1   3   1   0 426   0   3   0   0   4]
 [  9   2   0   8   2 312   8   2   9   2]
 [  5   0   0   0   3   2 397   0   5   1]
 [  1   1   7   3   4   1   0 396   1   7]
 [  4   9   3   2   3   0   4   0 367   5]
 [  4   1   1   6  16   3   0   7   2 384]]
For class 0: 
Average Precision(AP): 0.934043, 
Average Recall(AR): 0.966960, 
F1 of AP and AR: 0.950216 


For class 1: 
Average Precision(AP): 0.933610, 
Average Recall(AR): 0.982533, 
F1 of AP and AR: 0.957447 


For class 2: 
Average Precision(AP): 0.941489, 
Average Recall(AR): 0.903061, 
F1 of AP and AR: 0.921875 


For class 3: 
Average Precision(AP): 0.936620, 
Average Recall(AR): 0.890625, 
F1 of AP and AR: 0.913043 


For class 4: 
Average Precision(AP): 0.924078, 
Average Recall(AR): 0.972603, 
F1 of AP and AR: 0.947720 


For class 5: 
Average Precisi

In [10]:
def restore_model1(input_checkpoint, 
                               test_data, test_label):
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', 
                                       clear_devices=True)
    # graph = tf.get_default_graph()  # 获得默认的图
    # input_graph_def = graph.as_graph_def()  # 返回一个序列化的图代表当前的图
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)  # 恢复图并得到数据
        
        graph = tf.get_default_graph()
        x = graph.get_tensor_by_name("x-input:0")
        y = graph.get_tensor_by_name("y-input:0")
        prediction = graph.get_tensor_by_name("predict_output:0")
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_data, y: test_label}))
        
        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored"))
        saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored") + ".json", as_text=True)
        
                
        confusion_matrix = tf.contrib.metrics.confusion_matrix(
            tf.argmax(y, 1), 
            tf.argmax(prediction, 1)
        )
        cm = confusion_matrix.eval({x: test_data, 
                                 y: test_label}
                                   )
        print(cm)
        return cm

In [11]:
cm_restore = restore_model1("CNN_model/cnn_model", tstd, tstl)
performance_metrics(cm_restore)

INFO:tensorflow:Restoring parameters from CNN_model/cnn_model


Accuracy: 0.91355085


[[430   2   3   3   1   5   5   1   2   2]
 [  0 436   0   5   1   3   4   5   3   1]
 [  5   4 345  11   2   1   3  10  11   0]
 [  3   5   8 394   1  18   3   4   6   6]
 [  0   0   3   1 397   0   9   5   4  19]
 [  4   1   1   9   4 311   9   1   8   6]
 [  2   2   2   0   4   1 396   1   4   1]
 [  3   0   9   2   2   2   0 390   1  12]
 [  6  12  10   7   5   5   5   1 341   5]
 [  4   2   1   6  13   1   1  13   3 380]]
For class 0: 
Average Precision(AP): 0.940919, 
Average Recall(AR): 0.947137, 
F1 of AP and AR: 0.944018 


For class 1: 
Average Precision(AP): 0.939655, 
Average Recall(AR): 0.951965, 
F1 of AP and AR: 0.945770 


For class 2: 
Average Precision(AP): 0.903141, 
Average Recall(AR): 0.880102, 
F1 of AP and AR: 0.891473 


For class 3: 
Average Precision(AP): 0.899543, 
Average Recall(AR): 0.879464, 
F1 of AP and AR: 0.889391 


For class 4: 
Average Precision(AP): 0.923256, 
Average Recall(AR): 0.906393, 
F1 of AP and AR: 0.914747 


For class 5: 
Average Precisi